<a href="https://colab.research.google.com/github/samitha278/VLM-gamma/blob/main/repro_gemma.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math


## Gemma Config

In [4]:
class GemmaConfig():

    def __init__(self,vocab_size,n_embd,n_hidden,n_head,n_kv_head,n_layer,head_dim=256,max_pos_embds=8192,rms_norm_eps=1e-6,rope_theta=1000.0,attention_bias=False,attention_dropot=0.0,pad_token_id=None):

        self.vocab_size = vocab_size
        self.n_embd = n_embd
        self.n_hidden = n_hidden
        self.n_head = n_head
        self.n_kv_head = n_kv_head
        self.n_layer = n_layer
        self.head_dim = head_dim
        self.max_pos_embds = max_pos_embds
        self.rms_norm_eps = rms_norm_eps
        self.rope_theta = rope_theta
        self.attention_bias = attention_bias
        self.attention_dropot = attention_dropot
        self.pad_token_id = pad_token_id

## VLMGamma Cofig

In [2]:
class VLMGammaConfig():

    def __init__(self,ignore_index=-100,img_token_index=256000,vocab_size=257152,projection_dim=2048,n_embd=2048,pad_token_id=None):
        super().__init__()

        self.siglip_config = SigLipViTConfig()
        self.gemma_config = GemmaConfig()

        self.ignore_index = ignore_index
        self.img_token_index = img_token_index
        self.vocab_size - vocab_size
        self.projection_dim = projection_dim
        self.n_embd = n_embd
        self.pad_token_id = pad_token_id




## VLMGamma

In [3]:
class VLMGamma(nn.Module):

    def __init__(self,config):
        super().__init__()

        self.config = config

        self.vision_encoder = SigLipViT(config.siglip_config)
        self.projector = VLProjector(config)

        self.vocab_size = config.vocab_size
        self.gemma = GemmaCausalLM(config.gemma_config)




    def tie_weights(self):
        return self.transformer_decoder.tie_weights()



    def forward(self,tokens,images,attention_mask,kv_cache):

        input_embds = self.transformer_decoder.get_input_embds()(tokens)

        vit_out_embds = self.vision_encoder(images)
        proj_vit_out_embds = self.projector(vit_out_embds)

        input_embds_merged = self.merge_text_img(tokens,proj_vit_out_embds,input_embds,attention_mask,kv_cache)


        outputs = self.gemma(attention_mask,tokens,input_embds_merged,kv_cache)
